In [1]:
!python -m venv C:\GreenGIS\NearMissDash
!C:\GreenGIS\NearMissDash\Scripts\activate.bat
!PS C:\GreenGIS\NearMissDash\Scripts\Activate.ps1

      PID    PPID    PGID     WINPID   TTY         UID    STIME COMMAND
    10836       1   10836      10836  ?         197610 09:41:46 /usr/bin/ps


In [16]:
#import packages
from flask import Flask, render_template, request, jsonify
import folium
from IPython.display import HTML, display
import json
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
import io
import smtplib
from email.mime.text import MIMEText
from waitress import serve
import requests

#from your_database_module import search_function

#name app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'greenstreets'

#setup JSONBin.io 
JSONBIN_API_URL = 'https://api.jsonbin.io/v3/b/66ccd0c7e41b4d34e4259f46'
JSONBIN_ACCESS_KEY = '$2a$10$maXqFJ/sWO8DyZWVnrSXJOU/SvBX3.1Plo1Sg/kbzVV.4OrLe0ZU2'

#tell flask to read home page
@app.route('/')
def index(): 
    return render_template('apphome.html')

#tell flask to read about page
@app.route('/about')
def about(): 
    return render_template('about.html')

#tell flask to read contact page
@app.route('/contact')
def conact(): 
    return render_template('contact.html')

#tell flask to read contact page
@app.route('/defs')
def defs(): 
    return render_template('defs.html')

#tell flask to read dashboard page
@app.route('/dashboard')
def dashboard():
    return render_template('dashboard.html')


#tell flask how to add event to JSONBin
@app.route('/add-feature', methods=['POST'])
def add_feature():
    new_feature = request.json  # Get the new feature from the request body
    
    # Fetch the current data from JSONBin
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch data from JSONBin.io'}), response.status_code

    # Add the new feature to the existing data
    data = response.json()['record']
    data['features'].append(new_feature)

    # Update the JSONBin with the new data
    update_response = requests.put(JSONBIN_API_URL, headers=headers, json=data)

    if update_response.status_code == 200:
        return jsonify(update_response.json()), 200
    else:
        return jsonify({'error': 'Failed to update data in JSONBin.io'}), update_response.status_code
#tell flask how to show map from JSONBin data
@app.route('/get-map-data', methods=['GET'])
def get_map_data():
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)
    if response.status_code == 200:
        return jsonify(response.json()), 200
    else:
        return jsonify({'error': 'Failed to fetch data from JSONBin.io'}), response.status_code



#tell flask how to submit contact form
def send_email(subject, sender, recipients, text_body):
    msg = MIMEText(text_body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ", ".join(recipients)
    try:
        with smtplib.SMTP_SSL('smtp.live.com', 465) as server:
            server.login(os.getenv('EMAIL_USER'), os.getenv('EMAIL_PASS'))
            server.sendmail(sender, recipients, msg.as_string())
    except Exception as e:
        print(f"Failed to send email: {e}")
#tell flask to submit form, success message and reroute home
@app.route('/submit', methods=['POST'])
def submit():
    name = request.form['name']
    email = request.form['email']
    message = request.form['message']
    subject = "New Form Submission"
    sender = os.getenv('EMAIL_USER')
    recipients = ["egreenwell@planrva.org"]
    text_body = f"Name: {name}\nEmail: {email}\nMessage: {message}"
    send_email(subject, sender, recipients, text_body)
    # Return JSON response
    return jsonify({"message": "Thank you for your request."})


#tell flask run app
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/bootstrap/css/bootstrap-theme.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/bootstrap-slider/css/bootstrap-slider.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/stylesheet.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/PlanRVA_White.png HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/nearmiss1.png HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 11:12:39] "GET /static/arrow2.png HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 11:12:42] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2024 11:12:42] "GET /static/bootstrap-slider/css/bootstrap-slider.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Aug/2024 11:12:42] "GET /static/stylesheet.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 11:12:42] "GET /static/nearmiss1.png HTTP/1.1" 304 -
127